In [ ]:
import os
import sys
import requests
import pandas as pd

sys.path.append("..")
from models.cases import Case

from models.scraper import ScraperBase

In [ ]:

start_date = "03/06/2024"
end_date = "03/06/2024"

url = "https://jpwebsite.harriscountytx.gov/PublicExtracts/GetExtractData"

params = {
    "extractCaseType": "CR",
    "extract": "1",
    "court": "305",
    "casetype": "CRCIT,CRCOM",
    "format": "csv",
    "fdate": start_date,
    "tdate": end_date,
}

response = requests.get(
    url=url,
    params=params,
)
print(response.status_code)

In [ ]:
data = response.text
print(data)

In [ ]:
# Define the path to your CSV file
csv_file_temp_path = 'Temp/harriscountytx.csv'

# Split the path in directory and file name
directory = os.path.dirname(csv_file_temp_path)

# Check if the directory exists, create it if it doesn't
if not os.path.exists(directory):
    os.makedirs(directory)

with open(csv_file_temp_path, "w", encoding="utf-8") as f:
    f.write(data)

In [ ]:
# Get pydantic fields for Case
case_fields = Case.__fields__.keys()
print(case_fields)

In [ ]:
df = pd.read_csv(
    csv_file_temp_path,
    skip_blank_lines=True
)

In [ ]:
df.columns

In [ ]:
df[['Def Height Feet', 'Def Height Inches']]

In [ ]:
# Rename columns dict
field_mapping = {
    "Case Number": "case_id",
    "JP Court ID ": "court_id",
    "Citation Number ": "ticket",
    "Filed Date": "filing_date",
    "Offense Code": "charges",
    "Offense Description ": "description",
    "Statutory Cite 1 ": "case_desc",
    "Statutory Cite 2 ": "related_cases",  # Assuming related laws might imply related cases
    "Arresting Agency Code ": "court_code",  # Closest match; may not be perfect
    "Arresting Agency Name ": "court_desc",
    "Officer Code": "source",  # Assuming the officer's code is a source identifier
    "Officer Name": "prosecuting_atty",  # Close approximation; officer acting as complainant
    "Plea Code": "plea_andpayind",
    "Plea Date ": "plea_date",
    "Disposition Code ": "status",
    "Disposition Date ": "case_status",
    "Judgment Date": "case_date",
    "Def First Name ": "first_name",
    "Def Middle Name": "middle_name",
    "Def Last Name": "last_name",
    "Def Suffix ": "suffix",  # No direct match; added as 'suffix'
    "Def Home Add Line 1": "address_line_1",
    "Def Home Add Line 2": "address_seq_no",  # Approximation; sequence no. may refer to multi-line addresses
    "Def Home Add City": "address_city",
    "Def Home Add State ": "address_state_code",
    "Def Home Add ZIP 1 ": "address_zip",
    # 'Def Home Add ZIP 2 ': Not directly mapped; possibly combine with 'Def Home Add ZIP 1 ' if needed
    "Def Work Add Line 1": "formatted_party_address",  # Assuming work address is a formatted party address
    # Remaining work address fields are not directly mapped due to lack of direct correspondence
    "Def Date of Birth": "birth_date",
    "Def SPN Number": "pidm",  # Assuming SPN number can serve as a unique identifier akin to PIDM
    "Def SPN Pointer": "custom",  # No direct match; 'custom' for miscellaneous data
    "Def Height Feet": "height",  # No direct match; added as 'height'
    "Def Height Inches": "height",  # Combine with 'Def Height Feet' for full height in one field
    "Def Weight": "weight",  # No direct match; added as 'weight'
    "Def Gender ": "gender",
    "Def Race": "race",  # No direct match; added as 'race'
    "Total Fines Assessed ": "fine",
    "Total Fines Due": "balance_due",  # Assuming closest match for dues
    # The fields related to costs, payments, bond details, and hearing details are not directly mapped due to lack of direct correspondence or because they are overly specific without a clear counterpart.
    "Next Hearing Date": "court_date",
    "Next Hearing Time": "court_time",
    # 'Next Hearing Desc', 'Last Open Warr Type', 'Last Open Warr Date': Not directly mapped due to lack of clear counterparts
}

In [ ]:
df = df.rename(columns=field_mapping)

df.columns

In [ ]:
df["court_date"]

In [ ]:
# 07/26/2024"08:30"
df["court_date_parsed"] = pd.to_datetime(df["court_date"], format='%m/%d/%Y"%H:%M"')

In [ ]:
df["court_date"] = df["court_date_parsed"].dt.date
df["court_time"] = df["court_date_parsed"].dt.time

In [ ]:
df.head(5)

In [ ]:
df.describe()

In [ ]:
res = requests.get(url="https://jpwebsite.harriscountytx.gov/PublicExtracts/search.jsp")
if res.status_code != 200:
    raise Exception("Failed to get search page")
from bs4 import BeautifulSoup
parser = BeautifulSoup(res.text, 'html.parser')
courts = parser.find("select", id="court").find_all("option")
courts = [
    {
        "court_id": court["value"],
        "name": court.text.strip()
    }
    for court in courts
]
courts

In [38]:
import sys
sys.path.append("..")

import requests
import pandas as pd
from models.cases import Case
from models.scraper import ScraperBase
from bs4 import BeautifulSoup
from datetime import date, datetime, time
from tempfile import NamedTemporaryFile
from rich.console import Console
from models.leads import Lead
from models.scraper import ScraperBase
from rich.progress import Progress


console = Console()


class TXHarrisCountyScraper(ScraperBase):
    field_mapping = {
        "Case Number": "case_id",
        "JP Court ID ": "court_id",
        "Citation Number ": "ticket",
        "Filed Date": "filing_date",
        "Offense Code": "charges",
        "Offense Description ": "description",
        "Statutory Cite 1 ": "case_desc",
        "Statutory Cite 2 ": "related_cases",  # Assuming related laws might imply related cases
        "Arresting Agency Code ": "court_code",  # Closest match; may not be perfect
        "Arresting Agency Name ": "court_desc",
        "Officer Code": "source",  # Assuming the officer's code is a source identifier
        "Officer Name": "prosecuting_atty",  # Close approximation; officer acting as complainant
        "Plea Code": "plea_andpayind",
        "Plea Date ": "plea_date",
        "Disposition Code ": "status",
        "Disposition Date ": "case_status",
        "Judgment Date": "case_date",
        "Def First Name ": "first_name",
        "Def Middle Name": "middle_name",
        "Def Last Name": "last_name",
        "Def Suffix ": "suffix",  # No direct match; added as 'suffix'
        "Def Home Add Line 1": "address_line_1",
        # "Def Home Add Line 2": "address_seq_no",  # Approximation; sequence no. may refer to multi-line addresses REMOVED; 
        "Def Home Add Line 2": "address_line_2",
        "Def Home Add City": "address_city",
        "Def Home Add State ": "address_state_code",
        "Def Home Add ZIP 1 ": "address_zip",
        # 'Def Home Add ZIP 2 ': Not directly mapped; possibly combine with 'Def Home Add ZIP 1 ' if needed
        "Def Work Add Line 1": "formatted_party_address",  # Assuming work address is a formatted party address
        # Remaining work address fields are not directly mapped due to lack of direct correspondence
        "Def Date of Birth": "birth_date",
        "Def SPN Number": "pidm",  # Assuming SPN number can serve as a unique identifier akin to PIDM
        "Def SPN Pointer": "custom",  # No direct match; 'custom' for miscellaneous data
        "Def Height Feet": "height_feet",  # No direct match; added as 'height'
        "Def Height Inches": "height_inches",  # Combine with 'Def Height Feet' for full height in one field
        "Def Weight": "weight",  # No direct match; added as 'weight'
        "Def Gender ": "gender",
        "Def Race": "race",  # No direct match; added as 'race'
        "Total Fines Assessed ": "fine",
        "Total Fines Due": "balance_due",  # Assuming closest match for dues
        # The fields related to costs, payments, bond details, and hearing details are not directly mapped due to lack of direct correspondence or because they are overly specific without a clear counterpart.
        "Next Hearing Date": "court_date",
        "Next Hearing Time": "court_time",
        # 'Next Hearing Desc', 'Last Open Warr Type', 'Last Open Warr Date': Not directly mapped due to lack of clear counterparts
    }
    def get_courts(self):
        self.search_url = "https://jpwebsite.harriscountytx.gov/PublicExtracts/search.jsp"
        res = requests.get(url=self.search_url)
        if res.status_code != 200:
            raise Exception("Failed to get search page")
        parser = BeautifulSoup(res.text, 'html.parser')
        courts = parser.find("select", id="court").find_all("option")
        return [
            {
                "court_id": str(court["value"]),
                "name": court.text.strip()
            }
            for court in courts
        ]

    def scraper_single_court(self, court_id, start_date, end_date):
        params = {
            "extractCaseType": "CR",
            "extract": "1",
            "court": court_id,
            "casetype": "CRCIT,CRCOM",
            "format": "csv",
            "fdate": start_date,
            "tdate": end_date,
        }

        response = requests.get(
            url=self.url,
            params=params,
        )

        with NamedTemporaryFile(delete=False, mode="w", encoding="utf-8") as f:
            f.write(response.text)
            filepath = f.name
            cases_dicts = self.extract_cases(filepath)        
            console.log(f"Extracted {len(cases_dicts)} cases")

            # with progress
            with Progress() as progress:
                task = progress.add_task("[red]Inserting cases...", total=len(cases_dicts))
                for case_dict in cases_dicts:
                    case_id = case_dict.get("case_id")
                    if self.check_if_exists(case_id):
                        console.log(f"Case {case_id} already exists. Skipping...")
                        continue
                    case = Case(**case_dict)
                    lead = Lead(**case_dict)
                    self.insert_case(case)
                    self.insert_lead(lead)

                    progress.update(task, advance=1)

    def extract_cases(self, filepath):
        try:
            df = pd.read_csv(
                filepath,
                skip_blank_lines=True
            )
            if df.empty:
                return []
        except pd.errors.EmptyDataError:
            return []
        except pd.errors.ParserError:
            return []
        
        df = df[list(self.field_mapping.keys())].rename(columns=self.field_mapping)

        df["court_date_parsed"] = pd.to_datetime(df["court_date"], format='%m/%d/%Y"%H:%M"')

        df["court_date"] = df["court_date_parsed"].dt.date
        df["court_time"] = df["court_date_parsed"].dt.time

        df["height"] = df["height_feet"].astype(str) + "'" + df["height_inches"].astype(str)

        cases_dicts = df.to_dict(orient="records")
        cases_dicts = [{k: (None if pd.isna(v) else v) for k, v in record.items()} for record in cases_dicts]            
        for cases_dict in cases_dicts:
            cases_dict["related_cases"] = [cases_dict["related_cases"]] if isinstance(cases_dict["related_cases"], str) else cases_dict["related_cases"]
            
            cases_dict["prosecuting_atty"] = None # I set this to None for now, not sure if this is True/False for default values
            cases_dict["fine"] = {
                "default_courtcost": 0,
                "total_amount": cases_dict["fine"],
                "total_vbfineamount": cases_dict["fine"],
            }
            cases_dict["ticket"] = {} # Push this as empty dict
            cases_dict["court_date"] = datetime.combine(cases_dict["court_date"], time()) if cases_dict["court_date"] else None
            cases_dict["court_time"] = str(cases_dict["court_time"])
            cases_dict["charges"] = [
                {
                    "description": cases_dict["charges"],
                }
            ]

        return cases_dicts

    def scrape(self, search_parameters):
        start_date = search_parameters.get("start_date")
        end_date = search_parameters.get("end_date")

        if not start_date:
            start_date = date.today().strftime("%m/%d/%Y")
        if not end_date:
            end_date = date.today().strftime("%m/%d/%Y")

        self.url = "https://jpwebsite.harriscountytx.gov/PublicExtracts/GetExtractData"

        # Get the list of courts
        courts_list = self.get_courts()

        # Loop for each court in the list of courts
        for court in courts_list:
            court_id = court.get("court_id")
            self.scraper_single_court(court_id, start_date, end_date)

In [39]:
txscraper = TXHarrisCountyScraper()
txscraper.scrape({"start_date": "03/06/2024", "end_date": "03/06/2024"})

[21:14:24] Extracted 69 cases                                                                      ]8;id=698809;file:///tmp/ipykernel_838161/552833638.py\552833638.py]8;;\:]8;id=206658;file:///tmp/ipykernel_838161/552833638.py#102\102]8;;\

Output()

           Extracted 30 cases                                                                      ]8;id=952880;file:///tmp/ipykernel_838161/552833638.py\552833638.py]8;;\:]8;id=820396;file:///tmp/ipykernel_838161/552833638.py#102\102]8;;\

Output()

           Extracted 100 cases                                                                     ]8;id=41079;file:///tmp/ipykernel_838161/552833638.py\552833638.py]8;;\:]8;id=328146;file:///tmp/ipykernel_838161/552833638.py#102\102]8;;\

...

Output()

           Extracted 47 cases                                                                      ]8;id=60656;file:///tmp/ipykernel_838161/552833638.py\552833638.py]8;;\:]8;id=277730;file:///tmp/ipykernel_838161/552833638.py#102\102]8;;\

Output()

[21:14:30] Extracted 0 cases                                                                       ]8;id=760306;file:///tmp/ipykernel_838161/552833638.py\552833638.py]8;;\:]8;id=611391;file:///tmp/ipykernel_838161/552833638.py#102\102]8;;\

Output()